In [187]:
key_const = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [188]:
import mido as md
import py_midicsv

raw_song = py_midicsv.midi_to_csv('chopin/chpn-p8.mid')
raw_song2 = py_midicsv.midi_to_csv('chopin/chpn-p21.mid')
raw_song3 = py_midicsv.midi_to_csv('chopin/Silent_Night_-_Pentatonic_ver..mid')

raw_song = [e.strip().split(",") for e in raw_song]
raw_song2 = [e.strip().split(",") for e in raw_song2]
raw_song3 = [e.strip().split(",") for e in raw_song3]
for k in raw_song:
    print(k)

['0', ' 0', ' Header', ' 1', ' 8', ' 480']
['1', ' 0', ' Start_track']
['1', ' 0', ' Title_t', ' "Prelude No. 8"']
['1', ' 0', ' Copyright_t', ' "Copyright \\251 1997 by Bernd Krueger http://www.piano-midi.de"']
['1', ' 0', ' Text_t', ' "Frederic Chopin"']
['1', ' 0', ' Text_t', ' "Molto agitato"']
['1', ' 0', ' Text_t', ' "Fertiggestellt am 6.5.97\\012"']
['1', ' 0', ' Text_t', ' "Korrekturen am 27.8.98\\012"']
['1', ' 0', ' Text_t', ' "Anpassungen an Digitalpiano am 18.12.98\\012"']
['1', ' 0', ' Text_t', ' "Update am 11.4.99\\012"']
['1', ' 0', ' Text_t', ' "Normierung: 23.12.2002\\012"']
['1', ' 0', ' Text_t', ' "Update am 10.6.2010\\012"']
['1', ' 0', ' Text_t', ' "Update am 20.2.2014\\012"']
['1', ' 0', ' Text_t', ' "Dauer: 1:31 Minuten\\012"']
['1', ' 0', ' SMPTE_offset', ' 96', ' 0', ' 3', ' 0', ' 0']
['1', ' 0', ' Time_signature', ' 4', ' 2', ' 24', ' 8']
['1', ' 0', ' Key_signature', ' 3', ' "major"']
['1', ' 0', ' Tempo', ' 666667']
['1', ' 80', ' Tempo', ' 662983']
['1', ' 

['2', ' 36000', ' Note_on_c', ' 0', ' 77', ' 0']
['2', ' 36060', ' Note_on_c', ' 0', ' 81', ' 51']
['2', ' 36120', ' Note_on_c', ' 0', ' 81', ' 0']
['2', ' 36120', ' Note_on_c', ' 0', ' 72', ' 63']
['2', ' 36180', ' Note_on_c', ' 0', ' 72', ' 0']
['2', ' 36180', ' Note_on_c', ' 0', ' 74', ' 55']
['2', ' 36240', ' Note_on_c', ' 0', ' 74', ' 0']
['2', ' 36240', ' Note_on_c', ' 0', ' 73', ' 63']
['2', ' 36300', ' Note_on_c', ' 0', ' 73', ' 0']
['2', ' 36300', ' Note_on_c', ' 0', ' 71', ' 60']
['2', ' 36360', ' Note_on_c', ' 0', ' 71', ' 0']
['2', ' 36420', ' Note_on_c', ' 0', ' 80', ' 60']
['2', ' 36480', ' Note_on_c', ' 0', ' 80', ' 0']
['2', ' 36540', ' Note_on_c', ' 0', ' 73', ' 52']
['2', ' 36600', ' Note_on_c', ' 0', ' 73', ' 0']
['2', ' 36600', ' Note_on_c', ' 0', ' 68', ' 60']
['2', ' 36660', ' Note_on_c', ' 0', ' 68', ' 0']
['2', ' 36660', ' Note_on_c', ' 0', ' 71', ' 51']
['2', ' 36720', ' Note_on_c', ' 0', ' 71', ' 0']
['2', ' 36720', ' Note_on_c', ' 0', ' 69', ' 61']
['2', ' 36

['4', ' 21840', ' Note_on_c', ' 0', ' 56', ' 0']
['4', ' 21840', ' Note_on_c', ' 0', ' 46', ' 56']
['4', ' 22080', ' Note_on_c', ' 0', ' 46', ' 0']
['4', ' 22080', ' Note_on_c', ' 0', ' 56', ' 64']
['4', ' 22092', ' Control_c', ' 0', ' 64', ' 0']
['4', ' 22135', ' Control_c', ' 0', ' 64', ' 127']
['4', ' 22160', ' Note_on_c', ' 0', ' 56', ' 0']
['4', ' 22160', ' Note_on_c', ' 0', ' 46', ' 58']
['4', ' 22240', ' Note_on_c', ' 0', ' 46', ' 0']
['4', ' 22240', ' Note_on_c', ' 0', ' 50', ' 64']
['4', ' 22320', ' Note_on_c', ' 0', ' 50', ' 0']
['4', ' 22320', ' Note_on_c', ' 0', ' 39', ' 58']
['4', ' 22560', ' Note_on_c', ' 0', ' 39', ' 0']
['4', ' 22560', ' Note_on_c', ' 0', ' 54', ' 67']
['4', ' 22575', ' Control_c', ' 0', ' 64', ' 0']
['4', ' 22609', ' Control_c', ' 0', ' 64', ' 127']
['4', ' 22640', ' Note_on_c', ' 0', ' 54', ' 0']
['4', ' 22640', ' Note_on_c', ' 0', ' 46', ' 60']
['4', ' 22720', ' Note_on_c', ' 0', ' 46', ' 0']
['4', ' 22720', ' Note_on_c', ' 0', ' 51', ' 63']
['4', ' 

# Load data (Note, Key, TimeSignature)

In [189]:
import os
import pandas as pd

songs = []
keys = []
time_sig =[]
for filename in os.listdir('chopin'):
    
    
    song = py_midicsv.midi_to_csv("chopin/" + filename)
    song = [e.strip().split(", ") for e in song]
    
    divider = int(song[0][5])

    df = pd.DataFrame(song)
    
    keys.append(  (  int(df[ df[2] == "Key_signature"][3].values[0]), df[ df[2] == "Key_signature"][4].values[0])  )
    time_sig.append( int(df[ df[2] == "Time_signature"][3].values[0]) )
    
    df = df[ (df[2]=='Note_on_c') ]
    
    df[4] = df[4].astype(int)
    df[1] = df[1].astype(int)
    df[1] = df[1]/divider
    df = df.sort_values(by = [4,1] , ascending=True)
    df = df[[1,4] ]
    df = df.reset_index(drop=True)
    songs.append(df)
    
songs[4].head(100)

,1,4
0,168.0,30
1,168.5,30
2,210.0,30
3,210.5,30
4,216.0,30
...,...,...
95,78.5,42
96,81.0,42
97,81.5,42
98,102.0,42


# Calculate Note Diff

In [190]:
diff = []
for i in range(len(songs)):

    rng = range(0, len(songs[i]), 2)
    rng2 = range(1, len(songs[i]), 2)
    
    start_notes = songs[i].iloc[rng]
    stop_notes = songs[i].iloc[rng2]
    
    start_notes = start_notes.reset_index(drop=True)
    stop_notes = stop_notes.reset_index(drop=True)
    
     
    diff.append(stop_notes[1].subtract(start_notes[1]).to_frame()  )
type(diff[0])

pandas.core.frame.DataFrame

# Restruct Dataframe 

In [222]:
notes = []
for i in range(len(songs)):
    
    rng = range(0, len(songs[i]), 2)
    start_notes = songs[i].iloc[rng]
    
    start_notes = start_notes.reset_index(drop=True)
    diff[i].rename(columns = {1:2}, inplace = True)
    
    result = pd.concat([start_notes, diff[i]], axis=1, join='inner')
    result.rename(columns = {1:'start', 4:'pitch', 
                              2:'duration'}, inplace = True)
    result = result.sort_values(by = ['start','duration'] , ascending=True)
    notes.append(result)
notes[0].tail(10)

,start,pitch,duration
193,31.666667,65,0.166667
176,31.833333,64,0.166667
224,31.833333,67,0.166667
15,32.000000,36,2.000000
42,32.166667,43,1.833333
50,32.333333,48,1.666667
65,32.500000,52,1.500000
92,32.666667,55,1.333333
135,32.833333,60,1.166667
177,33.000000,64,1.000000


# Contruct Melody Matrix

In [347]:
import numpy as np
import matplotlib.pyplot as plt
songs_chart = []

for i in range(len(notes) ):
    index = 0
    song_chart = []
    for bar in range (1,int(notes[i].iloc[-1]['start'])+1):
        arr = np.zeros([128, 97])
        while notes[i].iloc[index]['start'] < bar:
            start = (notes[i].iloc[index]['start']-bar+1)*96
            stop = start + (notes[i].iloc[index]['duration'])*96
            print(bar,round(start,5),round(stop,5) )
            #print((notes[i].iloc[index]['start']-bar+1)*96,(notes[i].iloc[index]['duration'])*96,bar)
            
            arr[int(notes[i].iloc[index]['pitch'])][int(start):int(stop)] = np.ones(arr[int(notes[i].iloc[index]['pitch'])][int(start):int(stop)].shape)
            #print(arr2.shape)
            index+=1
        
        #showA(arr,i,bar)
    print(i)
       
#         print('cut')

#     print('cut_song')


1 0.2 16.0
1 16.0 32.0
1 16.0 80.0
1 32.0 48.0
1 32.0 48.0
1 48.0 64.0
1 48.0 80.0
1 64.0 80.0
1 80.0 96.0
1 80.0 96.0
2 0.0 16.0
2 16.0 32.0
2 16.0 80.0
2 32.0 48.0
2 32.0 48.0
2 48.0 64.0
2 48.0 80.0
2 64.0 80.0
2 80.0 96.0
2 80.0 96.0
3 0.0 16.0
3 16.0 32.0
3 16.0 80.0
3 32.0 48.0
3 32.0 48.0
3 48.0 64.0
3 48.0 80.0
3 64.0 80.0
3 80.0 96.0
3 80.0 96.0
4 0.0 16.0
4 16.0 32.0
4 16.0 80.0
4 32.0 48.0
4 32.0 48.0
4 48.0 64.0
4 48.0 80.0
4 64.0 80.0
4 80.0 96.0
4 80.0 96.0
5 0.0 16.0
5 16.0 32.0
5 16.0 80.0
5 32.0 48.0
5 32.0 48.0
5 48.0 64.0
5 48.0 80.0
5 64.0 80.0
5 80.0 96.0
5 80.0 96.0
6 0.0 16.0
6 16.0 32.0
6 16.0 80.0
6 32.0 48.0
6 32.0 48.0
6 48.0 64.0
6 48.0 80.0
6 64.0 80.0
6 80.0 96.0
6 80.0 96.0
7 0.0 16.0
7 16.0 32.0
7 16.0 80.0
7 32.0 48.0
7 32.0 48.0
7 48.0 64.0
7 48.0 80.0
7 64.0 80.0
7 80.0 96.0
7 80.0 96.0
8 0.0 16.0
8 16.0 32.0
8 16.0 80.0
8 32.0 48.0
8 32.0 48.0
8 48.0 64.0
8 48.0 80.0
8 64.0 80.0
8 80.0 96.0
8 80.0 96.0
9 0.0 16.0
9 16.0 32.0
9 16.0 80.0
9 32.0 48.0
9

45 0.0 48.0
45 48.0 96.0
45 48.0 96.0
46 0.0 48.0
46 0.0 48.0
46 0.0 48.0
46 48.0 72.0
46 48.0 96.0
46 72.0 84.0
46 84.0 96.0
47 0.0 48.0
47 0.0 48.0
47 48.0 91.2
47 48.0 96.0
48 0.0 48.0
48 0.0 48.0
48 48.0 96.0
48 48.0 96.0
49 0.0 48.0
49 0.0 48.0
49 0.0 48.0
49 48.0 96.0
49 48.0 96.0
50 0.0 48.0
50 0.0 48.0
50 0.0 48.0
50 48.0 96.0
50 48.0 96.0
51 0.0 48.0
51 0.0 48.0
51 0.0 48.0
51 48.0 96.0
51 48.0 96.0
52 0.0 48.0
52 0.0 48.0
52 0.0 48.0
52 48.0 96.0
52 48.0 96.0
52 48.0 96.0
53 0.0 48.0
53 0.0 48.0
53 0.0 48.0
53 48.0 96.0
53 48.0 96.0
53 48.0 96.0
54 0.0 48.0
54 0.0 48.0
54 0.0 48.0
54 48.0 96.0
54 48.0 96.0
54 48.0 96.0
55 0.0 48.0
55 0.0 48.0
55 0.0 48.0
55 48.0 72.0
55 72.0 84.0
55 84.0 96.0
56 0.0 48.0
56 0.0 48.0
56 48.0 91.2
56 48.0 96.0
57 0.0 48.0
57 0.0 48.0
57 48.0 96.0
57 48.0 96.0
58 0.0 48.0
58 0.0 48.0
58 0.0 48.0
58 48.0 72.0
58 48.0 96.0
58 72.0 84.0
58 84.0 96.0
59 0.0 48.0
59 0.0 48.0
59 48.0 91.2
59 48.0 96.0
60 0.0 48.0
60 0.0 48.0
60 48.0 96.0
60 48.0 96.0


53 0.0 24.0
53 0.0 36.0
53 0.0 36.0
53 0.0 36.0
53 24.0 36.0
53 24.0 36.0
53 24.0 36.0
53 48.0 72.0
53 48.0 72.0
53 48.0 72.0
53 48.0 84.0
53 48.0 84.0
53 48.0 84.0
53 72.0 84.0
53 72.0 84.0
53 72.0 84.0
54 0.0 24.0
54 0.0 24.0
54 0.0 24.0
54 0.0 36.0
54 0.0 36.0
54 0.0 36.0
54 24.0 36.0
54 24.0 36.0
54 24.0 36.0
54 48.0 72.0
54 48.0 72.0
54 48.0 84.0
54 48.0 84.0
54 48.0 84.0
54 72.0 84.0
54 72.0 84.0
55 0.0 48.0
55 0.0 48.0
55 0.0 48.0
55 0.0 48.0
55 0.0 48.0
55 48.0 72.0
55 48.0 84.0
55 48.0 84.0
55 48.0 84.0
55 48.0 84.0
55 48.0 84.0
55 72.0 84.0
55 72.0 84.0
56 0.0 24.0
56 0.0 24.0
56 0.0 36.0
56 0.0 36.0
56 0.0 36.0
56 0.0 36.0
56 0.0 36.0
56 24.0 36.0
56 24.0 36.0
56 48.0 96.0
56 48.0 96.0
56 48.0 96.0
56 48.0 96.0
56 48.0 96.0
56 48.0 96.0
57 0.0 24.0
57 0.0 24.0
57 0.0 36.0
57 0.0 36.0
57 0.0 36.0
57 0.0 36.0
57 24.0 36.0
57 24.0 36.0
57 48.0 72.0
57 48.0 72.0
57 48.0 84.0
57 48.0 84.0
57 48.0 84.0
57 48.0 84.0
57 48.0 84.0
57 72.0 84.0
57 72.0 84.0
58 0.0 24.0
58 0.0 24.0
58 

28 48.0 96.0
29 0.0 48.0
29 48.0 96.0
30 0.0 48.0
30 48.0 96.0
31 0.0 48.0
31 0.0 288.0
31 0.0 288.0
31 0.0 288.0
31 48.0 96.0
32 0.0 48.0
32 48.0 96.0
33 0.0 48.0
33 48.0 96.0
34 0.0 48.0
34 0.0 264.0
34 0.0 272.0
34 0.0 288.0
34 48.0 96.0
35 0.0 48.0
35 48.0 96.0
36 0.0 48.0
36 48.0 84.0
36 72.0 384.0
36 88.0 96.0
37 0.0 48.0
37 0.0 288.0
37 0.0 288.0
37 48.0 96.0
38 0.0 48.0
38 48.0 96.0
39 0.0 48.0
39 48.0 96.0
40 0.0 48.0
40 0.0 288.0
40 0.0 288.0
40 0.0 288.0
40 48.0 96.0
41 0.0 48.0
41 48.0 96.0
42 0.0 48.0
42 48.0 96.0
43 0.0 48.0
43 48.0 96.0
44 0.0 48.0
44 48.0 96.0
45 0.0 48.0
45 48.0 96.0
46 0.0 48.0
46 48.0 96.0
47 0.0 48.0
47 48.0 96.0
48 0.0 48.0
48 48.0 72.0
48 72.0 100.0
49 0.0 48.0
49 0.0 288.0
49 0.0 288.0
49 0.0 288.0
49 48.0 96.0
50 0.0 48.0
50 48.0 96.0
51 0.0 48.0
51 48.0 96.0
52 0.0 48.0
52 0.0 288.0
52 0.0 288.0
52 0.0 288.0
52 48.0 96.0
53 0.0 48.0
53 48.0 96.0
54 0.0 48.0
54 48.0 96.0
55 0.0 48.0
55 0.0 288.0
55 0.0 288.0
55 0.0 576.0
55 48.0 96.0
56 0.0 48.0

180 48.0 96.0
180 72.0 96.0
181 0.0 48.0
181 0.0 288.0
181 0.0 288.0
181 0.0 288.0
181 48.0 96.0
182 0.0 48.0
182 48.0 96.0
183 0.0 48.0
183 48.0 96.0
184 0.0 48.0
184 0.0 288.0
184 0.0 288.0
184 0.0 576.0
184 48.0 96.0
185 0.0 48.0
185 48.0 96.0
186 0.0 48.0
186 48.0 96.0
187 0.0 48.0
187 0.0 288.0
187 0.0 288.0
187 0.0 288.0
187 48.0 96.0
188 0.0 48.0
188 48.0 96.0
189 0.0 48.0
189 48.0 96.0
190 0.0 48.0
190 0.0 264.0
190 0.0 288.0
190 0.0 288.0
190 48.0 96.0
191 0.0 48.0
191 48.0 96.0
192 0.0 48.0
192 48.0 96.0
192 72.0 96.0
193 0.0 48.0
193 0.0 288.0
193 0.0 288.0
193 0.0 288.0
193 48.0 96.0
193 48.0 576.0
194 0.0 48.0
194 48.0 96.0
195 0.0 48.0
195 48.0 96.0
196 0.0 48.0
196 0.0 192.0
196 0.0 192.0
196 0.0 192.0
196 48.0 96.0
197 0.0 48.0
197 48.0 96.0
198 0.0 48.0
198 0.0 96.0
198 0.0 96.0
198 48.0 96.0
199 0.0 48.0
199 0.0 192.0
199 0.0 192.0
199 48.0 96.0
199 48.0 288.0
200 0.0 48.0
200 48.0 96.0
201 0.0 48.0
201 48.0 96.0
202 0.0 48.0
202 0.0 288.0
202 0.0 288.0
202 0.0 288.0


...

731 0.0 24.0
731 24.0 48.0
731 48.0 72.0
731 48.0 96.0
731 48.0 96.0
731 72.0 96.0
732 0.0 24.0
732 24.0 48.0
732 48.0 72.0
732 72.0 96.0
733 0.0 24.0
733 0.0 48.0
733 0.0 48.0
733 24.0 48.0
733 48.0 72.0
733 48.0 96.0
733 48.0 96.0
733 72.0 96.0
734 0.0 24.0
734 0.0 48.0
734 0.0 48.0
734 24.0 48.0
734 48.0 72.0
734 48.0 72.0
734 48.0 72.0
734 72.0 96.0
735 0.0 24.0
735 0.0 48.0
735 0.0 48.0
735 24.0 48.0
735 48.0 72.0
735 48.0 96.0
735 48.0 96.0
735 72.0 96.0
736 0.0 24.0
736 0.0 24.0
736 0.0 24.0
736 24.0 48.0
736 48.0 72.0
736 48.0 96.0
736 48.0 96.0
736 72.0 96.0
737 0.0 24.0
737 0.0 48.0
737 0.0 48.0
737 24.0 48.0
737 48.0 96.0
737 48.0 96.0
737 48.0 96.0
737 48.0 96.0
737 48.0 96.0
738 0.0 24.0
738 0.0 48.0
738 24.0 48.0
738 48.0 64.0
738 48.0 96.0
738 48.0 96.0
738 64.0 80.0
738 80.0 96.0
739 0.0 48.0
739 0.0 192.0
739 48.0 96.0
739 48.0 96.0
739 48.0 144.0
740 0.0 48.0
740 0.0 48.0
740 0.0 48.0
740 48.0 96.0
740 48.0 96.0
740 48.0 96.0
741 0.0 24.0
741 0.0 48.0
741 24.0 48.0
74

843 0.0 24.0
843 24.0 48.0
843 24.0 48.0
843 48.0 68.0
843 48.0 68.0
843 64.0 84.0
843 64.0 84.0
843 80.0 94.0
843 80.0 94.0
844 0.0 48.0
844 0.0 48.0
844 0.0 48.0
844 48.0 96.0
844 48.0 96.0
845 0.0 48.0
845 0.0 48.0
845 48.0 96.0
845 48.0 96.0
845 51.0 96.0
845 51.0 96.0
845 54.0 96.0
845 54.0 96.0
845 57.0 96.0
846 0.0 24.0
846 0.0 24.0
846 24.0 48.0
846 24.0 48.0
846 48.0 68.0
846 48.0 68.0
846 64.0 84.0
846 64.0 84.0
846 80.0 94.0
846 80.0 94.0
847 0.0 48.0
847 0.0 48.0
847 48.0 96.0
847 48.0 96.0
847 48.0 96.0
848 0.0 48.0
848 0.0 48.0
848 0.0 48.0
848 48.0 96.0
848 48.0 96.0
848 51.0 96.0
848 51.0 96.0
848 54.0 96.0
848 54.0 96.0
848 57.0 96.0
849 0.0 24.0
849 0.0 24.0
849 24.0 48.0
849 24.0 48.0
849 48.0 68.0
849 48.0 68.0
849 64.0 84.0
849 64.0 84.0
849 80.0 94.0
849 80.0 94.0
850 0.0 48.0
850 0.0 48.0
850 0.0 48.0
850 48.0 96.0
850 48.0 96.0
851 0.0 48.0
851 0.0 48.0
851 48.0 96.0
851 48.0 96.0
851 51.0 96.0
851 51.0 96.0
851 54.0 96.0
851 54.0 96.0
851 57.0 96.0
852 0.0 24.0

995 24.0 48.0
995 48.0 192.0
995 48.0 240.0
995 72.0 96.0
996 0.0 24.0
996 24.0 48.0
996 48.0 72.0
996 72.0 96.0
997 0.0 144.0
997 24.0 48.0
997 48.0 72.0
997 48.0 120.0
997 72.0 96.0
998 0.0 24.0
998 24.0 46.0
998 24.0 48.0
998 48.0 192.0
998 48.0 192.0
998 72.0 96.0
999 0.0 24.0
999 24.0 48.0
999 48.0 72.0
999 72.0 96.0
1000 0.0 24.0
1000 0.0 144.0
1000 24.0 48.0
1000 48.0 72.0
1000 72.0 96.0
1001 0.0 24.0
1001 24.0 48.0
1001 48.0 72.0
1001 48.0 144.0
1001 72.0 96.0
1002 0.0 24.0
1002 24.0 48.0
1002 48.0 72.0
1002 48.0 94.0
1002 72.0 96.0
1003 0.0 24.0
1003 0.0 48.0
1003 24.0 48.0
1003 48.0 72.0
1003 48.0 120.0
1003 72.0 96.0
1004 0.0 24.0
1004 24.0 48.0
1004 24.0 48.0
1004 48.0 72.0
1004 48.0 192.0
1004 72.0 96.0
1005 0.0 24.0
1005 24.0 48.0
1005 48.0 72.0
1005 72.0 96.0
1006 0.0 24.0
1006 0.0 96.0
1006 24.0 48.0
1006 48.0 72.0
1006 72.0 96.0
1007 0.0 24.0
1007 0.0 46.0
1007 24.0 48.0
1007 48.0 72.0
1007 48.0 192.0
1007 72.0 96.0
1008 0.0 24.0
1008 24.0 48.0
1008 48.0 72.0
1008 72.0

1114 0.0 48.0
1114 0.0 48.0
1115 48.0 96.0
1115 48.0 96.0
1115 48.0 96.0
1117 0.0 192.0
1117 0.0 192.0
1117 0.0 192.0
1117 0.0 192.0
1117 0.0 192.0
1117 0.0 192.0
1119 0.0 48.0
1119 0.0 48.0
1119 0.0 48.0
1119 0.0 48.0
1119 0.0 48.0
1119 0.0 48.0
1119 48.0 96.0
1119 48.0 96.0
1119 48.0 96.0
1119 48.0 96.0
1119 48.0 96.0
1119 48.0 96.0
1120 0.0 48.0
1120 0.0 48.0
1120 0.0 48.0
1120 0.0 48.0
1120 0.0 48.0
1120 0.0 48.0
1120 48.0 96.0
1120 48.0 96.0
1120 48.0 96.0
1120 48.0 96.0
1120 48.0 96.0
1120 48.0 96.0
1121 0.0 48.0
1121 0.0 48.0
1121 0.0 48.0
1121 0.0 48.0
1121 0.0 48.0
1121 0.0 48.0
1121 48.0 144.0
1121 48.0 144.0
1121 48.0 192.0
1121 48.0 192.0
1121 48.0 192.0
1121 48.0 192.0
1122 48.0 96.0
1122 48.0 96.0
1123 0.0 96.0
1123 0.0 96.0
1123 0.0 144.0
1123 0.0 144.0
1123 0.0 144.0
1123 0.0 144.0
1124 0.0 48.0
1124 0.0 48.0
1124 48.0 144.0
1124 48.0 144.0
1124 48.0 192.0
1124 48.0 192.0
1124 48.0 192.0
1124 48.0 192.0
1125 48.0 96.0
1125 48.0 96.0
1126 0.0 96.0
1126 0.0 96.0
1126 0.0 

189 0.0 47.8
189 0.0 95.8
189 48.0 95.8
190 95.8 95.8
190 95.8 95.8
191 0.0 191.8
191 0.0 191.8
191 0.0 191.8
191 0.0 192.0
192 0.0 47.8
192 48.0 95.8
193 0.0 0.0
193 0.0 95.8
193 47.8 95.8
193 47.8 95.8
193 48.0 95.8
193 48.0 95.8
194 0.0 0.0
194 0.0 287.8
194 0.0 287.8
194 0.0 287.8
196 95.8 95.8
197 0.0 95.8
197 0.0 95.8
197 0.0 191.8
197 0.0 191.8
197 0.0 287.8
198 0.0 95.8
198 0.0 95.8
199 0.0 95.8
199 0.0 95.8
200 0.0 47.8
200 0.0 191.8
200 0.0 191.8
200 48.0 95.8
201 0.0 0.0
201 0.0 47.8
201 47.8 95.8
201 48.0 95.8
201 48.0 95.8
202 0.0 95.8
202 0.0 95.8
202 0.0 95.8
202 0.0 95.8
202 0.0 95.8
203 0.0 191.8
203 0.0 191.8
203 0.0 191.8
204 0.0 47.8
204 0.0 47.8
204 48.0 95.8
204 48.0 95.8
205 0.0 95.8
205 0.0 95.8
206 0.0 95.8
206 0.0 95.8
206 0.0 96.0
206 0.0 96.0
206 0.0 143.8
207 47.8 96.0
207 47.8 96.0
207 48.0 95.8
207 48.0 95.8
207 48.0 95.8
208 0.0 95.8
208 95.8 95.8
208 95.8 95.8
209 0.0 0.0
209 0.0 0.0
209 0.0 191.8
209 95.8 191.8
209 95.8 191.8
210 0.0 95.8
210 0.0 95.8


In [220]:
index = 0
for bar in range (1,int(notes[3].iloc[-1]['start']) + 2):
        A = np.zeros([128, 97])
        while notes[3].iloc[index]['start'] < bar:
            print(notes[3].iloc[index]['start'], bar)
            index+=1
        print('cut')

0.0020833333333333333 1
0.0020833333333333333 1
0.0020833333333333333 1
0.0020833333333333333 1
0.0020833333333333333 1
0.25 1
0.5 1
0.5 1
0.5 1
0.5 1
0.75 1
cut
1.0 2
1.0 2
1.0 2
1.0 2
1.0 2
1.25 2
1.5 2
1.5 2
1.5 2
1.5 2
1.5 2
1.5 2
1.75 2
cut
2.0 3
2.0 3
2.0 3
2.0 3
2.0 3
2.25 3
2.5 3
2.5 3
2.5 3
2.5 3
2.5 3
2.75 3
cut
3.0 4
3.0 4
3.0 4
3.0 4
3.0 4
3.25 4
3.5 4
3.5 4
3.5 4
3.5 4
3.75 4
cut
4.0 5
4.0 5
4.0 5
4.0 5
4.0 5
4.25 5
4.5 5
4.5 5
4.5 5
4.5 5
4.5 5
4.5 5
4.75 5
cut
5.0 6
5.0 6
5.0 6
5.0 6
5.0 6
5.25 6
5.5 6
5.5 6
5.5 6
5.5 6
5.5 6
5.75 6
cut
6.0 7
6.0 7
6.0 7
6.0 7
6.0 7
6.25 7
6.5 7
6.5 7
6.5 7
6.5 7
6.75 7
cut
7.0 8
7.0 8
7.0 8
7.0 8
7.0 8
7.25 8
7.5 8
7.5 8
7.5 8
7.5 8
7.5 8
7.75 8
cut
8.0 9
8.0 9
8.0 9
8.0 9
8.0 9
8.25 9
8.5 9
8.5 9
8.5 9
8.5 9
8.5 9
8.75 9
cut
9.0 10
9.0 10
9.0 10
9.0 10
9.0 10
9.25 10
9.5 10
9.5 10
9.5 10
9.5 10
9.5 10
9.75 10
cut
10.0 11
10.0 11
10.0 11
10.0 11
10.0 11
10.25 11
10.5 11
10.5 11
10.5 11
10.75 11
cut
11.0 12
11.0 12
11.0 12
11.0 12
11.25 

99.0 100
99.0 100
99.25 100
99.5 100
99.5 100
99.5 100
99.5 100
99.75 100
cut
100.0 101
100.0 101
100.0 101
100.0 101
100.0 101
100.25 101
100.5 101
100.5 101
100.5 101
100.5 101
100.5 101
100.75 101
cut
101.0 102
101.0 102
101.25 102
101.5 102
101.5 102
101.5 102
101.5 102
101.5 102
101.75 102
cut
102.0 103
102.0 103
102.0 103
102.0 103
102.5 103
102.5 103
102.5 103
102.5 103
102.5 103
102.75 103
cut
103.0 104
103.0 104
103.0 104
103.0 104
103.0 104
103.25 104
103.5 104
103.5 104
103.5 104
103.5 104
103.75 104
cut
104.0 105
104.0 105
104.0 105
104.0 105
104.0 105
104.25 105
104.5 105
104.5 105
104.5 105
104.5 105
104.5 105
104.75 105
cut
105.0 106
105.0 106
105.0 106
105.0 106
105.25 106
105.5 106
105.5 106
105.5 106
105.5 106
105.75 106
cut
106.0 107
106.0 107
106.0 107
106.0 107
106.0 107
106.25 107
106.5 107
106.5 107
106.5 107
106.5 107
106.5 107
106.75 107
cut
107.0 108
107.0 108
107.0 108
107.0 108
107.25 108
107.5 108
107.5 108
107.5 108
107.5 108
107.5 108
107.75 108
cut
108.0

IndexError: single positional indexer is out-of-bounds

In [55]:
for k in raw_song3:
    print(k)

['0', ' 0', ' Header', ' 1', ' 5', ' 480']
['1', ' 0', ' Start_track']
['1', ' 0', ' Time_signature', ' 3', ' 2', ' 24', ' 8']
['1', ' 0', ' Key_signature', ' -1', ' "major"']
['1', ' 0', ' Tempo', ' 769231']
['1', ' 0', ' Control_c', ' 0', ' 121', ' 0']
['1', ' 0', ' Program_c', ' 0', ' 52']
['1', ' 0', ' Control_c', ' 0', ' 7', ' 100']
['1', ' 0', ' Control_c', ' 0', ' 10', ' 64']
['1', ' 0', ' Control_c', ' 0', ' 91', ' 30']
['1', ' 0', ' Control_c', ' 0', ' 93', ' 30']
['1', ' 0', ' MIDI_port', ' 0']
['1', ' 0', ' Note_on_c', ' 0', ' 60', ' 80']
['1', ' 0', ' Note_on_c', ' 0', ' 69', ' 80']
['1', ' 0', ' Note_on_c', ' 0', ' 60', ' 80']
['1', ' 719', ' Note_on_c', ' 0', ' 60', ' 0']
['1', ' 719', ' Note_on_c', ' 0', ' 69', ' 0']
['1', ' 719', ' Note_on_c', ' 0', ' 60', ' 0']
['1', ' 720', ' Note_on_c', ' 0', ' 62', ' 80']
['1', ' 720', ' Note_on_c', ' 0', ' 70', ' 80']
['1', ' 720', ' Note_on_c', ' 0', ' 62', ' 80']
['1', ' 959', ' Note_on_c', ' 0', ' 62', ' 0']
['1', ' 959', ' Note

In [226]:
for filename in os.listdir('chopin'):
    out=""
    song = py_midicsv.midi_to_csv("chopin/" + filename)
    song = [e.strip().split(", ") for e in song]
    df = pd.DataFrame(song)
    out += filename+" "
    out += df[ df[2] == "Key_signature"][3].values[0]
    print(out)

chpn-p1.mid 0
chpn-p10.mid 4
chpn-p11.mid 5
chpn-p12.mid 5
chpn-p13.mid 6
chpn-p14.mid -6
chpn-p15.mid -5
chpn-p16.mid -5
chpn-p17.mid -4
chpn-p18.mid -4
chpn-p19.mid -3
chpn-p2.mid 0
chpn-p20.mid -3
chpn-p21.mid -2
chpn-p22.mid -2
chpn-p23.mid -1
chpn-p24.mid -1
chpn-p3.mid 1
chpn-p4.mid 1
chpn-p5.mid 2
chpn-p6.mid 2
chpn-p7.mid 3
chpn-p8.mid 3
chpn-p9.mid 4
chpn_op10_e01.mid 0
chpn_op10_e05.mid -6
chpn_op10_e12.mid -3
chpn_op23.mid -2
chpn_op25_e1.mid -4
chpn_op25_e11.mid 0
chpn_op25_e12.mid -3
chpn_op25_e2.mid -4
chpn_op25_e3.mid -1
chpn_op25_e4.mid 0
chpn_op27_1.mid 4
chpn_op27_2.mid -5
chpn_op33_2.mid 2
chpn_op33_4.mid 2
chpn_op35_1.mid -5
chpn_op35_2.mid -6
chpn_op35_3.mid -5
chpn_op35_4.mid -5
chpn_op53.mid -4
chpn_op66.mid 4
chpn_op7_1.mid -2
chpn_op7_2.mid 0
chp_op18.mid -3
chp_op31.mid -5
Silent_Night_-_Pentatonic_ver..mid -1


In [327]:
def showA(A,a,b):
#     for i in range(len(arr ) ):
#         out = ""
#         for j in range (97):
#             out += str(int(arr[i][j])) + " "
#         print(out)

    X,Y = np.meshgrid(np.arange(A.shape[1]), np.arange(A.shape[0]))
    plt.scatter(X.flatten(), Y.flatten(), c=A.flatten())
    plt.savefig('plot/'+str(a)+"_"+str(b)+".png")
    #plt.show()

In [352]:
a = 1.9954
(5.01).is_integer()

False